In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_08 import *

## ConvNet

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), 1, c)

In [ ]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [ ]:
nfs = [8,16,32,16,10]

In [ ]:
def conv_layer(ni, nf, ks=3, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True),
        nn.BatchNorm2d(nf),
        GeneralRelu(**kwargs))

In [ ]:
opt_func = optim.Adam

In [ ]:
learn,run = get_learn_run(nfs, data, 0.1, conv_layer,
                          opt_func=opt_func, cbs=cbfs)

In [ ]:
%time run.fit(3, learn)

train: [0.5115361328125, tensor(0.8409, device='cuda:0')]
valid: [0.20876630859375, tensor(0.9368, device='cuda:0')]
train: [0.09640099609375, tensor(0.9707, device='cuda:0')]
valid: [0.14875955810546876, tensor(0.9560, device='cuda:0')]
train: [0.069162666015625, tensor(0.9786, device='cuda:0')]
valid: [0.09412249145507813, tensor(0.9708, device='cuda:0')]
CPU times: user 7.73 s, sys: 1.94 s, total: 9.67 s
Wall time: 9.67 s
